In [5]:
#최종 코드 / 정확도 출력 X
#랜덤포레스트, LR, 결정트리, KNN 피싱 파일 넣은 것
import numpy as np
import pandas as pd
from tqdm import tqdm

#모델 생성
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

#데이터 프레임으로 구성된 데이터 읽어오기
train_dataset = pd.read_csv('ld1.csv')

#훈련용 데이터셋 구성
print("Data Shape = ", np.shape(train_dataset))
X_train = np.array(train_dataset.iloc[:, 1:-1].values)
y_train = np.array(train_dataset.iloc[:, -1].values)


#개별 ML 모델을 위한 Classifier 생성
rf_clf = RandomForestClassifier(n_estimators = 100, random_state = 0)
lr_clf = LogisticRegression(solver = 'liblinear', random_state = 0)
dt_clf = DecisionTreeClassifier()
knn_clf = KNeighborsClassifier()

test_dataset = pd.read_csv('ld2.csv')
print("Data Shape = ", np.shape(test_dataset))
X_test = np.array(test_dataset.iloc[:, 1:-1].values)
y_test = np.array(test_dataset.iloc[:, -1].values)

#예측값을 기입할 열 추가
test_dataset['label'] = np.nan

#투표 분류
voting_clf = VotingClassifier(
   estimators = [('rf', rf_clf), ('lr', lr_clf), ('dt', dt_clf), ('knn', knn_clf)],
   voting='soft')

voting_clf = voting_clf.fit(X_train, y_train)

#새로운 데이터 입력 및 예측 값 저장
for i in tqdm(range(0,len(test_dataset))):
    X_new = np.array([test_dataset.iloc[i, 1:-1].values]) #url 없애서 할 때 [i, :-1]
    y_new = voting_clf.predict(X_new)
    test_dataset.iloc[i,-1] = y_new


print(accuracy_score(y_test, voting_clf))


result_dataset = test_dataset[['url','label']]

result_dataset.to_csv('result.csv')


Data Shape =  (7565, 13)
Data Shape =  (3168, 12)
100%|██████████| 3168/3168 [00:33<00:00, 95.83it/s]


TypeError: Expected sequence or array-like, got <class 'sklearn.ensemble._voting.VotingClassifier'>